<div class="alert alert-block alert-danger">
<b>NOTE:</b> Installation of libraries only needs to be run once at setup.
</div>

In [39]:
conda install pandas

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [40]:
conda install regex

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install pdfplumber

In [34]:
import pdfplumber
import pandas as pd
import regex as re
from collections import namedtuple

# PDF data extraction

In [25]:
Inv = namedtuple('Inv', 'Description Amount')

<div class="alert alert-block alert-info">
<b>Note:</b> Open the file in Google Chrome, then copy and paste the URL of the file into the input below
</div>

In [33]:
filename = input("Please paste URL for the BOFA statement here:")
print(filename)

Please paste URL for the BOFA statement here: file:///D:/Code/Python/Sage50AutomationTest/2009.pdf


file:///D:/Code/Python/Sage50AutomationTest/2009.pdf


In [4]:
invoice_url = filename[8:]

In [24]:
with pdfplumber.open(invoice_url) as pdf:
    # all_pages = pdf.pages[2]
    # text = all_pages.extract_text()
    all_text = '' # new line
    for pdf_page in pdf.pages:
        single_page_text = pdf_page.extract_text()
        # print(single_page_text)
        # separate each page's text with newline
        all_text = all_text + '\n' + single_page_text

<div class="alert alert-block alert-warning">
<b>Alert:</b> Change index2 for individual type of bank statement.
</div>

In [23]:
for row in all_text.split('\n'):
    # Change Withdrawals to match the correct format on type of bank statemetn
    if row == 'Withdrawals and other subtractions':
        index1 = all_text.index(row)
    # Total other subtractions to be replaced by Checks to eliminate checks
    if row.startswith('Total other subtractions'):
        index2 = all_text.index(row)
        text = all_text[index1:index2]

In [26]:
payments = re.compile(r'^\d+/\d+/\d+.*')

<div class="alert alert-block alert-info">
<b>Note:</b> List of withdrawals can be checked in the printout below.
</div>

In [8]:
for line in text.split('\n'):
    if payments.match(line):
        print(line)

08/23/21 PMNT SENT  0821 VENMO 8558124430   NY 55480771233602162274972 -2.00
08/23/21 PMNT SENT  0823 VENMO 8558124430   NY 55480771235602370914904 -6.20
09/08/21 PMNT SENT  0907 VENMO 8558124430   NY 55480771250602837881183 -38.39
09/13/21 PURCHASE   0911 Groupon, Inc. 312-288-6424 IL -13.00
09/15/21 PMNT SENT  0914 VENMO 8558124430   NY 55480771257602530475948 -10.33
09/15/21 PMNT SENT  0914 VENMO 8558124430   NY 55480771257602531985192 -11.19
08/23/21 Online Banking Transfer Conf# hnj6uhpv1; Qiu -90.00
08/25/21 Bank of America Credit Card Bill Payment -196.31
08/26/21 VANGUARD BUY     DES:INVESTMENT ID:706047130002019  INDN:ABRAM ZHENYU QIU        CO -250.00
09/03/21 Latitude Med Cen DES:Rent       ID:XXXXXXXXX  INDN:Abram Qiu               CO ID:1861072180 WEB -1,116.62
09/07/21 Bank of America Credit Card Bill Payment -347.50
09/10/21 VANGUARD BUY     DES:INVESTMENT ID:706047130002019  INDN:ABRAM ZHENYU QIU        CO -250.00
09/21/21 Bank of America Credit Card Bill Payment -254.4

In [9]:
for line in text.split('\n'):
    if payments.match(line):
        vend_date, *vend_des, vend_cost = line.split()
        vend_des = ' '.join(vend_des)
        # print(vend_des, vend_cost)

In [10]:
line_items = []
for line in text.split('\n'):
    if payments.match(line):
        vend_date, *vend_des, vend_cost = line.split()
        vend_des = ' '.join(vend_des)
        vend_cost = float(vend_cost.replace(',', ''))*-1
        line_items.append(Inv(vend_des, vend_cost))

In [12]:
df = pd.DataFrame(line_items)
df.head()

,Description,Amount
0,PMNT SENT 0821 VENMO 8558124430 NY 55480771233...,2.00
1,PMNT SENT 0823 VENMO 8558124430 NY 55480771235...,6.20
2,PMNT SENT 0907 VENMO 8558124430 NY 55480771250...,38.39
3,"PURCHASE 0911 Groupon, Inc. 312-288-6424 IL",13.00
4,PMNT SENT 0914 VENMO 8558124430 NY 55480771257...,10.33


# Data Preparation for Sage 50

In [28]:
des_check = df.Description.to_string(index=False)

<div class="alert alert-block alert-warning">
<b>Alert:</b> Change the file path to the spreadsheet of vendor codes as needed.
</div>

In [15]:
d = dict()
f = open("D:\Code\Python\Sage50AutomationTest\CodesDatabase_Test.csv")
for line in f:
    line = line.strip('\n')
    (key, val) = line.split(",")
    d[key] = val
del d['ï»¿Keyword']
print(d)

{'venmo': '12345', 'online banking transfer': '70000', 'credit card bill payment': '246000', 'rent': '50000'}


In [29]:
codes = []
for line in des_check.split('\n'):
    # print(line.lower())
    indicator = False
    for key in d:
        if key in line.lower():
            codes.append(d[key])
            indicator = True
    if indicator == False:
        codes.append('24900')

In [32]:
df = df.assign(vendor_code=codes)
df = df[['Description', 'vendor_code', 'Amount']]
df.head()

,Description,vendor_code,Amount
0,PMNT SENT 0821 VENMO 8558124430 NY 55480771233...,12345,2.00
1,PMNT SENT 0823 VENMO 8558124430 NY 55480771235...,12345,6.20
2,PMNT SENT 0907 VENMO 8558124430 NY 55480771250...,12345,38.39
3,"PURCHASE 0911 Groupon, Inc. 312-288-6424 IL",24900,13.00
4,PMNT SENT 0914 VENMO 8558124430 NY 55480771257...,12345,10.33


### CSV file will be downloaded in next step.

In [ ]:
df.to_csv('inv_lines.csv', index=False)